In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [2]:
import gurobipy
import numpy as np

In [17]:
model = gurobipy.Model()

x1 = model.addVar(name='x1', vtype=gurobipy.GRB.BINARY)
x2 = model.addVar(name='x2', vtype=gurobipy.GRB.BINARY)

model.setObjective(1*x2 + 2*x1, gurobipy.GRB.MAXIMIZE)

model.addConstr(x1 >= 3)
model.addConstr(x2 <= 5)
model.addConstr(x2 - x1 == 1)

model.update()

model.getA().toarray()

array([[ 1.,  0.],
       [ 0.,  1.],
       [-1.,  1.]])

In [20]:
[c.sense for c in model.getConstrs()]

['>', '<', '=']

In [18]:
model.getAttr('rhs')

[3.0, 5.0, 1.0]

In [19]:
model.getAttr('obj')

[2.0, 1.0]

In [36]:
from src.data import oracle, load_instance

instance = load_instance('../data/raw/97_9.jl')
T = instance['tamanho'][0]
J = instance['jobs'][0]

model = oracle(list(range(J)), instance)

A = model.getA().toarray()
b = np.array(model.getAttr('rhs'))
c = np.array(model.getAttr('obj'))

A.shape, b.shape, c.shape

((5067, 1746), (5067,), (1746,))

In [46]:
constraints_sense = np.array([c.sense for c in model.getConstrs()])

A[constraints_sense == '<'] *= -1
b[constraints_sense == '<'] *= -1

A_ineq = A[constraints_sense != '=']
b_ineq = b[constraints_sense != '=']
A_eq = A[constraints_sense == '=']
b_eq = b[constraints_sense == '=']

A = np.concatenate((A_ineq, A_eq), axis=0)
b = np.concatenate((b_ineq, b_eq), axis=0)

((5067, 1746), (5067,))